In [1]:
import pandas as pd
import yaml
import os
import subprocess

In [2]:
hosts = pd.read_csv("../common/hosts.csv")
supressed_columns = ['ip','user']

hdfs_hosts = hosts[(hosts.hostname == "pi-node8") | (hosts.hostname == "pi-node7") | (hosts.hostname == "pi-node5") ]
hdfs_hosts['ip'].to_csv(r'hdfs-ip.txt', header=False, index=None, sep=' ')


i386_hosts = hosts[(hosts.hostname == "nuc-02") ]
i386_hosts['ip'].to_csv(r'i386-ip.txt', header=False, index=None, sep=' ')



hosts['ip'].to_csv(r'hosts-ip.txt', header=False, index=None, sep=' ')


In [3]:
raspi_name_hosts = hosts[(hosts.hostname == "pi-node8")]
raspi_data_hosts = hosts[(hosts.hostname == "pi-node7") | (hosts.hostname == "pi-node5")]

namenode_ip = raspi_name_hosts.iloc[0]['ip']
namenode_config = 'hadoop-namenode:%(namenode_ip)s' %  {"namenode_ip": namenode_ip}
hosts_list = [namenode_config]


#for index, row in raspi_data_hosts.iterrows():
#    hostname='hdfs-data-' + row['hostname']
#    datanode_config='%(hostname)s:%(ip)s' %  {"hostname": hostname,"ip": row['ip'] }
#    hosts_list.append(datanode_config)
    


In [4]:
add_host_list=" "    
for host in hosts_list: 
    add_host_list = add_host_list +" --add-host=" +host + "  "
    
    
for index, row in i386_hosts.iterrows():
    print("Add file to HDFS:")
    docker_host = 'source /root/common/env.sh && export DOCKER_HOST=\"tcp://%(ip)s:2376\"' %  {"ip": row['ip']}
    command = "docker stop spark-test && docker rm spark-test && docker run  %(hostlist)s   -d --name spark-test --env SPARK_OPTS=\"--driver-memory=6g\" --env SPARK_WORKER_MEMORY=2g  -p 0.0.0.0:8080:8080 -p 0.0.0.0:8888:8888 -p 0.0.0.0:4040:4040 jupyter/all-spark-notebook" % {"hostlist": add_host_list}
    result = subprocess.check_output("bash -c \"%s && %s || :   \" " %(docker_host,command)  , shell=True, encoding='utf-8')
    print(result)


Add file to HDFS:
spark-test
spark-test
6187e8bcfd34d1e87906fa759063f8737758561b14cbb5d6f168bafd59b28eca



In [5]:
i386_hosts.drop(columns=supressed_columns)

,hostname,arch,gpu
5,nuc-02,x86_64,0


In [6]:
%%bash
echo "ip|id|image|ports|runnning" > containers.csv
source ../common/env.sh 
while read ip; do
    export DOCKER_HOST="tcp://$ip:2376"
    docker ps   --format "$ip|{{.ID}}|{{.Image}}|{{.Ports}}|{{.RunningFor}}"  >> containers.csv 2>/dev/null || :
done <hosts-ip.txt

In [7]:
containers = pd.read_csv("containers.csv", sep = '|')
pd.merge(containers, hosts, on='ip', how='inner').drop(columns=supressed_columns)

,id,image,ports,runnning,hostname,arch,gpu
0,fb1114ef5ff5,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,3 weeks ago,pi-node2,armv7l,0
1,fbdf7410748b,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,3 weeks ago,pi-node3,armv7l,0
2,8c84fec83032,tensorflow/tensorflow:1.12.0-gpu,"6006/tcp, 0.0.0.0:8888->8888/tcp",3 weeks ago,kx-Lenovo-H520g,x86_64,1
3,c5c1b27b0828,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,3 weeks ago,kx-Lenovo-H520g,x86_64,1
4,4f9ee299d149,amkay/sensor-exporter,0.0.0.0:9255->9255/tcp,3 weeks ago,kx-Lenovo-H520g,x86_64,1
5,1b947aee9a12,registry:2,0.0.0.0:5000->5000/tcp,13 months ago,kx-Lenovo-H520g,x86_64,1
6,f86073a60478,prom/prometheus:v2.12.0,0.0.0.0:9090->9090/tcp,3 weeks ago,nuc-01,x86_64,0
7,0e0406780847,amkay/sensor-exporter,0.0.0.0:9255->9255/tcp,3 weeks ago,nuc-01,x86_64,0
8,bfb8d0b8a9fa,prom/node-exporter:v0.18.0,0.0.0.0:9100->9100/tcp,3 weeks ago,nuc-01,x86_64,0
9,6187e8bcfd34,jupyter/all-spark-notebook,"0.0.0.0:4040->4040/tcp, 0.0.0.0:8080->8080/tcp...",12 seconds ago,nuc-02,x86_64,0
